In [ ]:
import os
from datetime import datetime
path="D:\\User_Data\\Desktop\\Python_Kriging\\"
os.chdir(path)
import pandas as pd
import numpy as np
def date_to_delta(x):#transform numerate date to datetime
    x=str(x)
    time=datetime.strptime(x,"%Y%m%d")
    return time
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import logging
FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT,filename='mylog.log', filemode='a')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("kriging.csv")
pollution_set=set(df["pol"])
tw_grid=pd.read_csv(path+"tw_grid_3km.csv")
gridx=tw_grid["x"]
gridy=tw_grid["y"]
for p in pollution_set:
    if not os.path.exists(path+p):
        os.makedirs(path+p)
    for v in ["avg","max"]:
        if not os.path.exists(path+"\\"+p+"\\"+v):
            os.makedirs(path+"\\"+p+"\\"+v)
        sub_df_1=df[["date","x","y",v,"poll","station"]]
        sub_df_1["date_delta"]=sub_df_1["date"].apply(date_to_delta)
        sub_df_1=sub_df_1.sort_values("date_delta",ascending=False)
        sub_df_1=sub_df_1[sub_df_1["date_delta"]>=datetime(1998,1,1)]
        sub_df_1=sub_df_1[sub_df_1["pol"]==p]
        date_delta=set(sub_df_1["date_delta"])
        for d in date_delta:
            sub_df_2=sub_df_1[sub_df_1["date_delta"]==d]
            sub_df_2=sub_df_2.dropna(how="any")
            if len(sub_df_2)>0:
                data=np.array(sub_df_2)
                try:
                    OK = OrdinaryKriging(
                        data[:, 1],
                        data[:, 2],
                        data[:, 3],
                        variogram_model=m,
                        verbose=False,
                        enable_plotting=False,)
                    z, ss = OK.execute("points",gridx,gridy)
                    sub_df_3=pd.DataFrame()
                    sub_df_3["x"]=gridx
                    sub_df_3["y"]=gridy
                    sub_df_3["predicted_value"]=z
                    sub_df_3["predicted_std"]=ss
                    sub_df_3.to_csv(path+"\\"+p+"\\"+v+"\\"+p+"_"+v+"_"+"_"+str(sub_df_2["date"].iloc[0])+"_"+"kriging.csv")
                except Exception as e:
                            logging.debug(p+"_"+v+"_"+m+"_"+str(sub_df_2["date"].iloc[0])+"is not properly estimated")
            else:
                    logging.debug(str(d)+" is empty")
                

                    

                
                
            
            
            
            
    
    print(datetime.now())
            
            
            
            
            
            
            
        
        
        
        
    
    


In [ ]:
import os
from datetime import datetime
path="D:\\User_Data\\Desktop\\Python_Kriging\\"
os.chdir(path)
import pandas as pd
import numpy as np
def date_to_delta(x):#transform numerate date to datetime
    x=str(x)
    time=datetime.strptime(x,"%Y%m%d")
    return time
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import logging
FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT,filename='mylog.log', filemode='a')
import warnings
warnings.filterwarnings('ignore')
from geopy.distance import geodesic
from datetime import timedelta
import logging
FORMAT = '%(asctime)s %(levelname)s: %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT,filename='mylog2.log', filemode='a')

def distance(x,y):
    target=(x,y)
    shen_out=(25.1272,121.8156)
    dist=geodesic(shen_out,target)#target location should put in first place
    return dist.km
def dummy_for_distance(x):
    if x<=50:
        return 1
    else:
        return 0
def dummy_for_date(x):
    date=datetime.strptime(x,"%Y-%m-%d")
    if date>datetime(2007,9,30):
        return 1
    else:
        return 0
def dummy_for_both(x,y):
    return x*y

In [ ]:
for p in ["PM2.5","NO2","SO2","PM10","NO","NOx","O3"]:
    for v in ["avg","max"]:
        print(p+" "+v+" "+"starting",datetime.now())
        starting_time=datetime(1998,1,1)
        starting_df=pd.DataFrame([[-1,-1,-1,-1,datetime(1980,1,1),-1,-1,-1,-1]]
                                 ,columns=['x', 'y', 'predicted_value', 'predicted_std',"date","distance",
                                           "dummy_for_distance","dummy_for_date","dummy_for_both"])
        while starting_time<datetime(2017,1,1):
            try:
                
                df_by_date=pd.read_csv(path+p+"\\"+v+"\\"+"linear"+"\\"+p+"_"+v+"_"+"linear"+"_"
                                        +str(starting_time)[:4]+str(starting_time)[5:7]+str(starting_time)[8:10]+"_kriging.csv"
                                        ,index_col=0)
                
                date=[str(starting_time)[:10]]*len(df_by_date)
                df_by_date["date"]=date
                df_by_date["distance"]=df_by_date.apply(lambda x: distance(x["y"],x["x"]),axis=1)
                df_by_date["dummy_for_distance"]=df_by_date["distance"].apply(dummy_for_distance)
                df_by_date["dummy_for_date"]=df_by_date["date"].apply(dummy_for_date)
                df_by_date["dummy_for_both"]=df_by_date.apply(lambda x:dummy_for_both(x["dummy_for_distance"],x["dummy_for_date"]),axis=1)
                starting_df=starting_df.append(df_by_date,ignore_index=True)
            except Exception as e:
                pass
            starting_time=starting_time+timedelta(days=1)
        starting_df.to_csv(p+"_"+v+"_"+"for_regression.csv")
        print("sucessful")
        print(datetime.now())
        
            
      


In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
pollution=["CO","NO","NO2","NOx","O3","PM10","PM2.5","SO2"]
for i in pollution:
    globals()[i]=i
    

In [ ]:
reg_df=pd.read_csv("CO_avg_for_regression.csv")

In [ ]:
columns=['x', 'y', 'predicted_value', 'predicted_std', 'date',
       'distance', 'dummy_for_distance', 'dummy_for_date', 'dummy_for_both']

In [ ]:
Y=test[["predicted_value"]]
X=test[["dummy_for_distance","dummy_for_date","dummy_for_both"]]

In [ ]:
model=sm.OLS(Y,X)
result=model.fit()

In [2]:
import os
from datetime import datetime
import pandas as pd
for p in ["PM2.5","NO2","SO2","PM10","NO","NOx","O3"]:
    for v in ["avg","max"]:
        print("start",datetime.now())
        starting_time=datetime(1998,1,1)
        starting_df=pd.DataFrame([[-1,-1,-1,-1,datetime(1980,1,1)]]
                                 ,columns=['x', 'y', 'predicted_value', 'predicted_std',"date"])
        path="D:\\User_Data\\Desktop\\Python_Kriging\\"+p+"\\"+v+"\\"+"linear"
        os.chdir(path)
        filelist=os.listdir()
        for f in filelist:
            subdf=pd.read_csv(path+"\\"+f,index_col=0)
            date=f.split("_")[3]
            subdf["date"]=date
            starting_df=pd.concat([starting_df,subdf],axis=0)
        starting_df.to_csv("D:\\User_Data\\Desktop\\Python_Kriging\\"+p+"_"+v+"_"+"combined_without_preprocessed.csv")
        print("end",datetime.now())


start 2021-01-28 09:35:56.249265
end 2021-01-28 12:30:16.732314
start 2021-01-28 12:30:16.735314
end 2021-01-28 15:19:43.051334
start 2021-01-28 15:19:43.051334
end 2021-01-28 18:01:44.342290
start 2021-01-28 18:01:44.342290
end 2021-01-28 20:51:01.270125
start 2021-01-28 20:51:01.270125
end 2021-01-28 23:33:50.225509
start 2021-01-28 23:33:50.241134
end 2021-01-29 02:15:35.923565
start 2021-01-29 02:15:35.923565
end 2021-01-29 04:55:59.763414
start 2021-01-29 04:55:59.763414
end 2021-01-29 07:38:15.044510
start 2021-01-29 07:38:15.060136
end 2021-01-29 10:21:57.212129
start 2021-01-29 10:21:57.212129
end 2021-01-29 13:03:22.023465
start 2021-01-29 13:03:22.026420
end 2021-01-29 15:45:26.721083
start 2021-01-29 15:45:26.723090
end 2021-01-29 18:32:03.288006
start 2021-01-29 18:32:03.288006
end 2021-01-29 21:35:09.277606
start 2021-01-29 21:35:09.277606
end 2021-01-30 00:24:09.118481


In [6]:
df1=pd.read_csv("D:\\User_Data\\Desktop\\Python_Kriging\\O3_max_combined_without_preprocessed.csv")

In [5]:
df2=pd.read_csv("D:\\User_Data\\Desktop\\Python_Kriging\\Nox_max_combined_without_preprocessed.csv")

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
